## 모듈 import

In [1]:
#모듈 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
from pycaret.classification  import *

# Warning
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")

# Scailing
from sklearn.preprocessing import PowerTransformer

# Label Encoding
from sklearn.preprocessing import LabelEncoder

print('Pandas : %s'%(pd.__version__))
print('Numpy : %s'%(np.__version__))
print('matplotlib : %s'%(matplotlib.__version__))
print('pycaret : %s'%(pycaret.__version__))
!python --version

Pandas : 1.3.2
Numpy : 1.19.5
matplotlib : 3.3.4
pycaret : 2.3.3
Python 3.8.8


사용한 버전  
Pandas : 1.3.2  
Numpy : 1.19.5  
matplotlib : 3.3.4  
pycaret : 2.3.3  
Python 3.8.8

## data 불러오기

In [2]:
# 데이터가 저장된 경로 설정
data_path = './daplatformers2021/'

# 데이터 불러오기
train = pd.read_csv(data_path + 'trainset.csv')
test = pd.read_csv(data_path + 'testset_final.csv')
submission = pd.read_csv(data_path + 'sample_submission_final.csv')

In [3]:
train = train.drop(columns=['매니저최초가입일', '매니저최초서비스일', '매니저주소'])

### 전처리

In [4]:
data = pd.concat([train,test]).reset_index(drop=True)

- 결측치 채움

In [5]:
# 결측치 0으로 채움
data['청소교육이수여부'] = data['청소교육이수여부'].fillna(0)
data['CS교육이수여부'] = data['CS교육이수여부'].fillna(0)
data['부재중여부'] = data['부재중여부'].fillna(0)
# 결측치 최빈값으로 채움
data['매니저이동방법'] = data['매니저이동방법'].fillna(data['매니저이동방법'].mode()[0])
data['매니저사용휴대폰'] = data['매니저사용휴대폰'].fillna(data['매니저사용휴대폰'].mode()[0])
data['고객가입일'] = data['고객가입일'].fillna(data['고객가입일'].mode()[0]) #데이터 바뀌고 test에 생긴 에러
data['결재형태'] = data['결재형태'].fillna(data['결재형태'].mode()[0]) #데이터 바뀌고 test에 생긴 에러

data['접수시각'] = data['접수시각'].fillna(data['접수시각'].mode()[0])
data['반려동물'] = data['반려동물'].fillna('없음')
# 결측치 판단하여 없음으로 채움
data['우선청소'] = data['우선청소'].fillna('없음')

### 평수 평균 확인 -> 31.xx -> 30평대로 채움
data['평수'] = data['평수'].fillna('30평대')

In [6]:
#test와 train의 구조를 맞춰줌
data['매니저생년월일'] = data['매니저생년월일'].astype('str').apply(lambda x: x[0:4])

- Make features

In [7]:
# 새로운 피처 만들기
data['매니저나이'] = data['매니저생년월일'].apply(lambda x: 2021-int(x))
data['매니저나이대'] = data['매니저나이'].apply(lambda x: f'{str(x)[0]}0대')
# 현재 월을 기준으로 가입한지 몇개월이 됐는지 계산
data['고객가입개월수_str'] = data['고객가입일'].apply(lambda x:str((2021-(int(x[0:4])+1))*12 + 12-int(x[5:7])+1 + 11) )
data['고객가입개월수_int'] = data['고객가입일'].apply(lambda x:(2021-(int(x[0:4])+1))*12 + 12-int(x[5:7])+1 + 11)

In [8]:
# 저장해놓기
data_SEQ = data.SEQ
# 인코딩을 위하여 제거 / 이후 나중에 병합함
y_data = data['매칭성공여부']
data = data.drop(columns = ['SEQ','매칭성공여부'])

- Encoding & Scailing

In [9]:
#범주형변수 수동으로 분리
data[['장기서비스여부','기존고객여부','부재중여부','쿠폰사용여부','매니저성별',
         'CS교육이수여부','청소교육이수여부','부재중서비스가능여부','추천인여부']
     ] = data[['장기서비스여부','기존고객여부','부재중여부','쿠폰사용여부','매니저성별',
         'CS교육이수여부','청소교육이수여부','부재중서비스가능여부','추천인여부'
     ]].astype('str')

# 범주형 변수와 수치형 변수를 분리
cat_features = data.select_dtypes(include=['object']).columns.to_list()
num_features = data.select_dtypes(exclude='object').columns.to_list() 

#수치형변수 scailing
data[num_features] = PowerTransformer(standardize=True).fit_transform(data[num_features])

#라벨인코딩
for f in cat_features:
    data[f] = LabelEncoder().fit_transform(data[f])

In [10]:
# 피처 선정
data = data.drop(columns=['현재회차', '기존고객여부', '쿠폰사용여부','매니저성별','접수일',
                              '고객가입개월수_str','고객가입개월수_int'])

In [11]:
data.columns

Index(['접수시각', '장기서비스여부', '최초서비스일', '전체회차', '서비스일자', '서비스시작시간', '서비스종료시간',
       '결재형태', '서비스주소', '주거형태', '평수', '고객가입일', '반려동물', '부재중여부', '우선청소',
       '매니저생년월일', '매니저사용휴대폰', '매니저이동방법', '근무가능지역', 'CS교육이수여부', '청소교육이수여부',
       '부재중서비스가능여부', '추천인여부', '매니저나이', '매니저나이대'],
      dtype='object')

In [12]:
#위에서 떼어놨던 피처 병합
data = pd.concat([data_SEQ, data],axis=1)
data = pd.concat([data, y_data],axis=1);data.head(3)

,SEQ,접수시각,장기서비스여부,최초서비스일,전체회차,서비스일자,서비스시작시간,서비스종료시간,결재형태,서비스주소,주거형태,평수,고객가입일,반려동물,부재중여부,우선청소,매니저생년월일,매니저사용휴대폰,매니저이동방법,근무가능지역,CS교육이수여부,청소교육이수여부,부재중서비스가능여부,추천인여부,매니저나이,매니저나이대,매칭성공여부
0,T06420,379,1,56,-0.650410,84,11,3,1,48,1,2,6,6,0,3,3,2,0,3,0,0,1,0,1.824058,4,0.0
1,T15430,379,1,56,-0.131658,103,1,6,1,48,1,2,18,6,0,3,23,2,0,6,1,1,1,0,-1.237119,2,0.0
2,T23790,379,1,342,1.998486,432,11,3,3,48,1,3,202,6,0,3,17,2,0,6,0,0,0,0,-0.458846,3,0.0


- Data split

In [13]:
#train과 test를 분리
train = data.iloc[:23009,:]
test = data.iloc[23009:,:]
test = test.drop(columns = ['매칭성공여부']).reset_index(drop=True)

## Modeling with pycaret

- Model setting

In [15]:
# SEQ 지워주기
train = train.drop(columns=['SEQ'])

In [16]:
cl = setup(train, 
            preprocess = False, # True-> Feature Engineering을 추가로 진행해 Predict가 불가능해진다.
            train_size = 0.999,  # 학습할 train의 수
            target = '매칭성공여부', # target값
            silent = True, # 엔터 누르지 않아도 자동으로 돌아감
            use_gpu = True, # GPU가 있으면 사용: 특정 경우에서 CPU보다 느릴 수 있음
            session_id = 42,
            fold_shuffle = True
            )

,Description,Value
0,session_id,42
1,Target,매칭성공여부
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(23009, 26)"
5,Missing Values,False
6,Numeric Features,25
7,Categorical Features,0
8,Transformed Train Set,"(22985, 25)"
9,Transformed Test Set,"(24, 25)"


In [17]:
top = compare_models(sort='AUC', fold=5, n_select = 1, include=['rf'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9661,0.9738,0.7006,0.9267,0.7978,0.7797,0.7889,1.0340


- Create Model

In [18]:
#rf -> 튜닝하면 성능 떨어짐
rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9743,0.9834,0.7909,0.9305,0.8550,0.8411,0.8444
1,0.9696,0.9763,0.7318,0.9360,0.8214,0.8051,0.8123
2,0.9696,0.9794,0.7455,0.9213,0.8241,0.8077,0.8131
3,0.9669,0.9739,0.7000,0.9390,0.8021,0.7845,0.7946
4,0.9687,0.9730,0.7364,0.9205,0.8182,0.8013,0.8073
5,0.9713,0.9876,0.7489,0.9371,0.8325,0.8170,0.8231
6,0.9678,0.9744,0.7123,0.9341,0.8083,0.7911,0.7997
7,0.9735,0.9765,0.7682,0.9441,0.8471,0.8328,0.8381
8,0.9630,0.9699,0.6682,0.9245,0.7757,0.7561,0.7680
9,0.9721,0.9788,0.7545,0.9432,0.8384,0.8234,0.8295


- Fit & predict

In [19]:
model = rf
pred_holdout = predict_model(model, test)

In [20]:
#pycaret에서는 score이 label을 맞출 확률이기때문에 output을 제출 양식에 맞게 바꿔줌
pred = pd.Series([pred_holdout['Score'][i] if pred_holdout['Label'][i] =='1.0' else (1-pred_holdout['Score'][i]) for i in range(len(pred_holdout['Score']))]
)

In [21]:
submission['pred'] = pred

### Make submission 

In [22]:
fname = 'rf_0.87398.csv'
submission.to_csv(fname, index=False)
print("'{}' is ready to submit." .format(fname))

'rf_0.87398.csv' is ready to submit.


# END